# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [9]:
!"{sys.executable}" -m pip install scikit-learn

     ---------------------------------------- 7.6/7.6 MB 884.1 kB/s eta 0:00:00
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


     ---------------------------------------- 40.2/40.2 MB 1.0 MB/s eta 0:00:00



You should consider upgrading via the 'C:\Users\Jorin Eggers\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [10]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [12]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [32]:

class StrToBytes:
    def __init__(self, fileobj):
        self.fileobj = fileobj
    def read(self, size):
        return self.fileobj.read(size).encode()
    def readline(self, size=-1):
        return self.fileobj.readline(size).encode()


def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    with open(filename, 'r') as f:
        data = pickle.load(StrToBytes(f))
    target = [i] * len(data)
    return data, target

In [36]:
# load all the data
all_data = []
all_target = []

for i, class_name in enumerate(classes):
    data, target = load_pose_data(i)
    all_data+=data
    all_target+=target
    
print('total number of data', len(all_data))

total number of data 222


In [41]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
test_data = permutation[n_training_data:]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [43]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [70]:
all_data = np.array(all_data)
all_target = np.array(all_target)

X_train = all_data[training_data]
y_train = all_target[training_data]

X_test = all_data[test_data]
y_test = all_target[test_data]

clf.fit(X_train, y_train)

SVC(C=100.0, gamma=0.001)

### predicting

In [80]:
expected = y_train
predicted = clf.predict(X_train)

In [81]:
expected, predicted

(array([ 2,  9, 10,  0,  8,  0,  2,  9,  6,  9,  4,  1, 10,  4,  6,  8,  7,
         0,  2,  1,  6,  8,  2,  8, 10, 10, 10, 10,  0, 10,  8, 10,  7,  3,
         3, 10, 10, 10,  2,  1, 10, 10,  4,  6, 10,  1,  6,  2,  4, 10,  8,
        10,  8,  4,  1, 10,  5,  1,  8,  2, 10,  3,  3,  0,  2,  8,  7,  9,
        10, 10,  8,  8,  3,  9,  0,  3,  2,  0, 10, 10,  0, 10, 10,  9,  6,
         4,  8,  2, 10, 10,  8,  8,  6,  5,  0,  0,  1,  7,  2,  2,  8,  1,
         0,  2,  4,  1,  0,  9,  2, 10, 10, 10,  8,  2, 10,  5,  5,  7,  0,
         0,  9,  2, 10,  6, 10,  1, 10,  2,  4,  0,  2, 10,  6, 10, 10,  3,
         1,  4,  7,  2,  1,  1,  2,  0,  6, 10,  6,  8,  6,  0,  8, 10,  6,
        10, 10]),
 array([ 2,  9, 10,  0,  8,  0,  2,  9,  6,  9,  4,  1, 10,  4,  6,  8,  7,
         0,  2,  1,  6,  8,  2,  8, 10, 10, 10, 10,  0, 10,  8, 10,  7,  3,
         3, 10, 10, 10,  2,  1, 10, 10,  4,  6, 10,  1,  6,  2,  4, 10,  8,
        10,  8,  4,  1, 10,  5,  1,  8,  2, 10,  3,  3,  0,  2,  8,  7

In [82]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [83]:
#expected = []
#predicted = []
# YOUR COD

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         9
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00        18
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00        40

    accuracy                           1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155


Confusion matrix:
[[17  0  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [85]:
expected = []
predicted = []
# YOUR CODE HERE

expected = y_test
predicted = clf.predict(X_test)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         6
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         7
           7       1.00      0.80      0.89         5
           8       1.00      1.00      1.00         8
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00        12

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.99        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [89]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [91]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, "rb"))
clf2.predict(all_data[-1].reshape(1,-1)), all_target[-1]

(array([10]), 10)